In [1]:
from collections import ChainMap

import plotly.express as px
import plotly.graph_objects as go
import polars as pl
from sqlmodel import Session, SQLModel, create_engine, select

from convergence_games.db.models import (
    AdventuringGroup,
    AllocationResult,
    CommittedAllocationResult,
    Compensation,
    ContentWarning,
    Game,
    GameContentWarningLink,
    GameGenreLink,
    Genre,
    Person,
    PersonAdventuringGroupLink,
    SessionPreference,
    System,
    Table,
    TableAllocation,
    TimeSlot,
)

# Aims

Answer the following:
- What is the 'path' of each attendee WRT attending sessions
- What makes a game popular?
  - GM?
  - Genre?
  - System?
- How fair is the game selection algorithm?

In [2]:
engine = create_engine("sqlite:///prod.db")
SQLModel.metadata.create_all(engine)

In [3]:
def make_df(every_result, prefix_by_type=False):
    if prefix_by_type:
        return pl.DataFrame(
            [
                dict(
                    ChainMap(
                        *[
                            {}
                            if x is None
                            else {"{}-{}".format(x.__class__.__name__, k): v for k, v in x.model_dump().items()}
                            for x in a
                        ]
                    )
                )
                for a in every_result
            ]
        )
    else:
        return pl.DataFrame([dict(ChainMap(*[{} if x is None else x.model_dump() for x in a])) for a in every_result])

In [ ]:
# All Games
with Session(engine) as session:
    games = session.exec(select(Game, System).join(System, Game.system_id == System.id)).all()
    games_df = make_df(games, prefix_by_type=True)
games_df = games_df.select(
    pl.col("Game-title").alias("Game"),
    pl.col("System-name").alias("System"),
).sort("System")
games_df.write_csv("games.csv")
unique_systems = games_df["System"].unique().to_list()
print(unique_systems)

In [ ]:
# GMs whose games got played
with Session(engine) as session:
    gms = session.exec(
        select(TableAllocation, Game, Person, CommittedAllocationResult, System)
        .join(Person, Game.gamemaster_id == Person.id)
        .join(TableAllocation, TableAllocation.game_id == Game.id)
        .outerjoin(CommittedAllocationResult, CommittedAllocationResult.table_allocation_id == TableAllocation.id)
        .join(System, Game.system_id == System.id)
    ).all()
    gms = make_df(gms, prefix_by_type=True)
unique_gms = (
    gms.unique(pl.col("Game-title", "Person-name", "Person-email", "TableAllocation-time_slot_id"))
    .select(
        pl.col("Game-title").alias("title"),
        pl.col("System-name").alias("system_name"),
        pl.col("Person-name").alias("name"),
        pl.col("Person-email").alias("email"),
        pl.col("TableAllocation-time_slot_id").alias("time_slot_id"),
        pl.col("CommittedAllocationResult-id").is_not_null().alias("played"),
    )
    .sort("name", "time_slot_id")
)
unique_gms.write_csv("gms_with_games.csv")
unique_gms[["system_name", "title"]].unique().sort(by="system_name").write_csv("game_title_and_system.csv")
unique_gms.select(pl.col("name", "email", "played")).group_by(pl.col("name", "email")).agg(pl.col("played").any()).sort(
    "played", "name", descending=(True, False)
).write_csv("gms.csv")
unique_gms

In [6]:
with Session(engine) as session:
    players_that_played_alien = session.exec(
        select(CommittedAllocationResult, Person)
        .join(AdventuringGroup, AdventuringGroup.id == CommittedAllocationResult.adventuring_group_id)
        .join(PersonAdventuringGroupLink, PersonAdventuringGroupLink.adventuring_group_id == AdventuringGroup.id)
        .join(Person, Person.id == PersonAdventuringGroupLink.member_id)
        .where(CommittedAllocationResult.table_allocation_id == 30070)
    ).all()
    df = make_df(players_that_played_alien)
df.select(pl.col("name", "email")).write_csv("alien_players.csv")

In [ ]:
with Session(engine) as session:
    every_result = session.exec(
        select(Person, AdventuringGroup, CommittedAllocationResult, TableAllocation, Game)
        .join(PersonAdventuringGroupLink, PersonAdventuringGroupLink.member_id == Person.id)
        .join(AdventuringGroup, PersonAdventuringGroupLink.adventuring_group_id == AdventuringGroup.id)
        .join(CommittedAllocationResult, CommittedAllocationResult.adventuring_group_id == AdventuringGroup.id)
        .join(TableAllocation, TableAllocation.id == CommittedAllocationResult.table_allocation_id)
        .join(Game, Game.id == TableAllocation.game_id)
    ).all()
    every_result = pl.DataFrame([dict(ChainMap(*[x.model_dump() for x in a])) for a in every_result])
every_result

In [ ]:
with Session(engine) as session:
    every_table_allocation = session.exec(select(TableAllocation)).all()
    every_table_allocation = pl.DataFrame([x.model_dump() for x in every_table_allocation])
    n_games_per_session = (
        every_table_allocation.filter(pl.col("game_id") != 0)
        .group_by("time_slot_id")
        .agg(pl.count("id").alias("total_games"))
        .sort("time_slot_id")
    )
n_games_per_session

In [ ]:
# Games running per time slot
every_result.filter(pl.col("game_id") != 0).group_by("time_slot_id").agg(
    pl.n_unique("game_id").alias("games_running"),
    pl.n_unique("adventuring_group_id").alias("groups"),
    pl.n_unique("email").alias("people"),
).with_columns(players=pl.col("people") - pl.col("games_running")).join(
    n_games_per_session, pl.col("time_slot_id")
).sort("time_slot_id")

In [ ]:
print()
every_result.n_unique("game_id"), every_result.n_unique("system_id")

In [ ]:
# attendance_df = (
#     df.select(pl.col("email").unique())
#     .with_columns(pl.lit([1, 2, 3, 4, 5]).alias("time_slot_id"))
#     .explode(pl.col("time_slot_id"))
#     .join(df, on=("email", "time_slot_id"), how="left")
#     .with_columns(
#         time_slot_id=pl.lit("session_") + pl.col("time_slot_id").cast(str),
#         checked_in=pl.col("checked_in").replace(None, False),
#     )
#     .pivot(on="time_slot_id", values="checked_in")
#     .with_columns(count=pl.sum_horizontal(pl.col("session_1", "session_2", "session_3", "session_4", "session_5")))
# )
# attendance_df
part_of_session_df = every_result.select(pl.col("email", "time_slot_id")).with_columns(playing=pl.lit(True))
part_of_session_df = (
    part_of_session_df.select(pl.col("email").unique())
    .with_columns(pl.lit([1, 2, 3, 4, 5]).alias("time_slot_id"))
    .explode(pl.col("time_slot_id"))
    .join(part_of_session_df, on=("email", "time_slot_id"), how="left")
    .with_columns(
        time_slot_id=pl.lit("session_") + pl.col("time_slot_id").cast(str),
        playing=pl.col("playing").replace(None, False),
    )
    .pivot(on="time_slot_id", values="playing")
    .with_columns(count=pl.sum_horizontal(pl.col("session_1", "session_2", "session_3", "session_4", "session_5")))
)
part_of_session_df.write_csv("participated_by_session.csv")
part_of_session_df

In [ ]:
for x in (
    part_of_session_df.with_columns(
        saturday=pl.col("session_1") | pl.col("session_2") | pl.col("session_3"),
        sunday=pl.col("session_4") | pl.col("session_5"),
        everything=pl.col("session_1")
        & pl.col("session_2")
        & pl.col("session_3")
        & pl.col("session_4")
        & pl.col("session_5"),
    )
    .with_columns(
        either=pl.col("saturday") | pl.col("sunday"),
        both=pl.col("saturday") & pl.col("sunday"),
    )
    .sum()
):
    if x.name in ("email", "count"):
        continue
    print(f"{x.name:15}{x.sum()}")

In [ ]:
# Games which ran at least once
every_result.filter(pl.col("game_id") != 0).group_by("game_id", "time_slot_id").first().select(
    pl.col("game_id", "time_slot_id", "title", "table_id")
).sort("game_id")

In [51]:
# Players that checked in at least once:
with Session(engine) as session:
    every_player_that_checked_in = session.exec(
        select(Person).where(
            select(PersonAdventuringGroupLink)
            .where(PersonAdventuringGroupLink.member_id == Person.id)
            .join(AdventuringGroup, AdventuringGroup.id == PersonAdventuringGroupLink.adventuring_group_id)
            .where(AdventuringGroup.checked_in)
            .exists()
        )
    ).all()
    players = pl.DataFrame([x.model_dump() for x in every_player_that_checked_in])
players.select(pl.col("name", "email")).sort(pl.col("email")).write_csv("checked_in_players.csv")

In [ ]:
# Which sessions was each person checked in for?
with Session(engine) as session:
    sessions_checked_in_for_each_player = session.exec(
        select(Person, AdventuringGroup)
        .join(PersonAdventuringGroupLink, PersonAdventuringGroupLink.member_id == Person.id)
        .join(AdventuringGroup, AdventuringGroup.id == PersonAdventuringGroupLink.adventuring_group_id)
        .where(AdventuringGroup.checked_in)
    ).all()

    df = make_df(sessions_checked_in_for_each_player).select(pl.col("email", "time_slot_id", "checked_in"))
attendance_df = (
    df.select(pl.col("email").unique())
    .with_columns(pl.lit([1, 2, 3, 4, 5]).alias("time_slot_id"))
    .explode(pl.col("time_slot_id"))
    .join(df, on=("email", "time_slot_id"), how="left")
    .with_columns(
        time_slot_id=pl.lit("session_") + pl.col("time_slot_id").cast(str),
        checked_in=pl.col("checked_in").replace(None, False),
    )
    .pivot(on="time_slot_id", values="checked_in")
    .with_columns(count=pl.sum_horizontal(pl.col("session_1", "session_2", "session_3", "session_4", "session_5")))
)
attendance_df

In [ ]:
for x in (
    attendance_df.with_columns(
        saturday=pl.col("session_1") | pl.col("session_2") | pl.col("session_3"),
        sunday=pl.col("session_4") | pl.col("session_5"),
        everything=pl.col("session_1")
        & pl.col("session_2")
        & pl.col("session_3")
        & pl.col("session_4")
        & pl.col("session_5"),
    )
    .with_columns(
        either=pl.col("saturday") | pl.col("sunday"),
        both=pl.col("saturday") & pl.col("sunday"),
    )
    .sum()
):
    if x.name in ("email", "count"):
        continue
    print(f"{x.name:15}{x.sum()}")

In [ ]:
with Session(engine) as session:
    every_system = session.exec(select(System)).all()
    every_genre = session.exec(select(Genre)).all()
    every_game = session.exec(select(Game)).all()
len(every_system), len(every_genre), len(every_game)

In [ ]:
unique_gms = set()
for game in every_game:
    unique_gms.add(game.gamemaster_id)
len(unique_gms)

In [ ]:
px.bar(
    attendance_df.group_by("count").agg(pl.len()).rename({"count": "number_of_sessions_attended", "len": "players"}),
    x="number_of_sessions_attended",
    y="players",
    text="players",
)

In [ ]:
# Most popular games
with Session(engine) as session:
    every_preference = session.exec(
        select(SessionPreference, TableAllocation, Game, System)
        .join(TableAllocation, SessionPreference.table_allocation_id == TableAllocation.id)
        .join(Game, TableAllocation.game_id == Game.id)
        .join(System, Game.system_id == System.id)
    ).all()
    every_preference = (
        pl.DataFrame([dict(ChainMap(*[x.model_dump() for x in a])) for a in every_preference])
        .select(pl.col("title", "game_id", "name", "preference", "adventuring_group_id"))
        .with_columns(
            title_and_system=pl.col("title") + " (" + pl.col("name") + ")",
            preference=pl.when(pl.col("preference") == 20).then(6).otherwise(pl.col("preference")),
        )
    )
    every_preference = every_preference.join(
        every_preference.group_by("game_id")
        .agg(
            pl.mean("preference").alias("mean_preference"),
            pl.count("preference").alias("n_preferences"),
        )
        .sort("mean_preference"),
        on="game_id",
    ).sort("mean_preference")
every_preference

In [ ]:
fig = px.box(every_preference, x="preference", y="title_and_system", height=3000)
fig.update_traces(boxmean=True)
fig

In [ ]:
from plotly.subplots import make_subplots

titles = []
preferences = []
for i, (game_id, game_df) in enumerate(every_preference.group_by("game_id", maintain_order=True)):
    title = game_df["title_and_system"][0]
    titles.append(title)
    preferences.append(game_df["preference"])
fig = make_subplots(rows=len(titles), cols=1, shared_xaxes=True, vertical_spacing=0.002, subplot_titles=titles)
for i, (title, preference) in enumerate(zip(titles, preferences)):
    fig.add_trace(
        go.Histogram(x=preference, name="count", texttemplate="%{y:.0f}", showlegend=False, histnorm="percent"),
        row=i + 1,
        col=1,
    )
fig.update_layout(height=150 * len(titles), title_text="Preference distribution per game")
fig

In [ ]:
import plotly.figure_factory as ff

# Popularity of games by number of times run
number_of_times_run = (
    every_result.filter(pl.col("game_id") != 0)
    .group_by("game_id")
    .agg(pl.n_unique("table_allocation_id").alias("n_times_run"))
    .sort("n_times_run")
)
group_labels = []
preferences = []
for n_times_run, df in every_preference.join(number_of_times_run, on="game_id").group_by("n_times_run"):
    group_labels.append(f"{n_times_run} times run")
    preferences.append(list(df["preference"]))
ff.create_distplot(preferences, group_labels=group_labels, histnorm="percent")

In [ ]:
from typing import Any

from sqlmodel import tuple_

from convergence_games.db.models import AdventuringGroupWithExtra


# What were the final scores of people that were allocated?
def make_df(every_result):
    return pl.DataFrame([dict(ChainMap(*[{} if x is None else x.model_dump() for x in a])) for a in every_result])


with Session(engine) as session:
    every_adventuring_group = session.exec(select(AdventuringGroup)).all()
    every_adventuring_group_with_extra = [AdventuringGroupWithExtra.model_validate(x) for x in every_adventuring_group]
    every_table_allocation = pl.DataFrame(session.exec(select(TableAllocation)).all())


every_table_allocation_by_time_slot: dict[int, list[int]] = {}

for time_slot, ta in every_table_allocation.group_by("time_slot_id"):
    every_table_allocation_by_time_slot[time_slot[0]] = list(ta["id"])

# adventuring_group_id : {table_allocation_id: preference}
every_preference_by_adventuring_group: list[dict[str, Any]] = []
for adventuring_group in every_adventuring_group_with_extra:
    assessed_table_allocation_ids = set()
    for preference in adventuring_group.session_preferences:
        assessed_table_allocation_ids.add(preference.table_allocation_id)
        every_preference_by_adventuring_group.append(
            {
                "adventuring_group_id": adventuring_group.id,
                "table_allocation_id": preference.table_allocation_id,
                "preference": preference.preference,
            }
        )

    for table_allocation_id in every_table_allocation_by_time_slot[adventuring_group.time_slot_id]:
        if table_allocation_id not in assessed_table_allocation_ids:
            every_preference_by_adventuring_group.append(
                {
                    "adventuring_group_id": adventuring_group.id,
                    "table_allocation_id": preference.table_allocation_id,
                    "preference": 3,
                }
            )

every_preference_by_adventuring_group = pl.DataFrame(every_preference_by_adventuring_group)
every_preference_by_adventuring_group = (
    every_preference_by_adventuring_group.group_by("adventuring_group_id")
    .agg(
        pl.mean("preference").alias("mean_preference"),
        pl.count("preference").alias("n_preferences"),
        pl.col("preference").alias("preferences"),
    )
    .with_columns(pl.col("preferences").list.sort(descending=True))
    .with_columns(pl.col("preferences").list.unique().list.sort(descending=True).alias("unique_preferences"))
    .sort("mean_preference")
)
every_preference_by_adventuring_group

In [ ]:
# # for adventuring_group_id, preferences in sorted_every_preference_by_adventuring_group.items():
# #     print(adventuring_group_id, len(preferences))

with Session(engine) as session:
    every_gm_group_id = session.exec(
        # select(PersonAdventuringGroupLink, AdventuringGroup, Game, TableAllocation)
        # .join(AdventuringGroup, AdventuringGroup.id == PersonAdventuringGroupLink.adventuring_group_id)
        # .join(Person, Person.id == PersonAdventuringGroupLink.member_id)
        # .join(Game, Game.gamemaster_id == Person.id)
        # .join(TableAllocation, TableAllocation.game_id == Game.id)
        select(TableAllocation, Person, PersonAdventuringGroupLink)
        .join(Game, TableAllocation.game_id == Game.id)
        .join(Person, Person.id == Game.gamemaster_id)
        .join(PersonAdventuringGroupLink, Person.id == PersonAdventuringGroupLink.member_id)
    )
    to_exclude = [
        (x["table_allocation_id"], x["adventuring_group_id"])
        for x in make_df(every_gm_group_id)
        .select(pl.col("id").alias("table_allocation_id"), "adventuring_group_id")
        .to_dicts()
    ]
    print(to_exclude)

    every_final_score = session.exec(
        select(CommittedAllocationResult, TableAllocation, SessionPreference)
        .join(TableAllocation, TableAllocation.id == CommittedAllocationResult.table_allocation_id)
        .outerjoin(
            SessionPreference,
            (SessionPreference.table_allocation_id == TableAllocation.id)
            & (SessionPreference.adventuring_group_id == CommittedAllocationResult.adventuring_group_id),
        )
        .where(~tuple_(TableAllocation.id, SessionPreference.adventuring_group_id).in_(to_exclude))
        # .where(~TableAllocation.game_id.in_([45, 70]))
    ).all()
    every_final_score = make_df(every_final_score).with_columns(pl.col("preference").replace(None, 3))
every_final_score = (
    every_final_score.join(every_preference_by_adventuring_group, on=("adventuring_group_id"))
    .with_columns(
        pl.struct(["unique_preferences", "preference"])
        .map_elements(lambda x: x["unique_preferences"].index(x["preference"]), return_dtype=pl.Int32)
        .alias("preference_index"),
    )
    .sort("time_slot_id")
)
print(every_final_score.columns)
px.pie(every_final_score, names="preference_index")  # , facet_col="time_slot_id")
# # everybodys_preferences_per_session

In [ ]:
with Session(engine) as session:
    players_with_adventuring_groups = session.exec(
        select(AdventuringGroup, Person)
        .join(PersonAdventuringGroupLink, PersonAdventuringGroupLink.member_id == Person.id)
        .join(AdventuringGroup, AdventuringGroup.id == PersonAdventuringGroupLink.adventuring_group_id)
    ).all()
    thing_to_join = []
    for adventuring_group, person in players_with_adventuring_groups:
        thing_to_join.append(
            {
                "adventuring_group_id": adventuring_group.id,
                "person_id": person.id,
            }
        )
    thing_to_join = pl.DataFrame(thing_to_join)
player_every_final_score = every_final_score.join(thing_to_join, on="adventuring_group_id", how="left").select(
    pl.col("time_slot_id", "preference_index", "person_id")
)
px.pie(player_every_final_score, names="preference_index", title="Rankings")
# players_with_adventuring_groups

In [ ]:
px.pie(
    player_every_final_score.group_by("person_id")
    .agg(
        pl.min("preference_index").alias("min_preference"),
        pl.max("preference_index").alias("max_preference"),
        pl.len().alias("n_games_played"),
    )
    .sort("n_games_played"),
    names="max_preference",
    facet_col="n_games_played",
    title="Worst ranking of game played",
)

In [ ]:
px.box(every_final_score, x="preference_index", y="mean_preference")